In [1]:
%cd /content
!git clone -b totoro5 https://github.com/camenduru/ComfyUI /content/TotoroUI
!git clone -b totoro5 https://github.com/camenduru/ComfyUI-AdvancedLivePortrait /content/TotoroUI/custom_nodes/TotoroUI-AdvancedLivePortrait
%cd /content/TotoroUI

!pip install torchsde dill ultralytics
!mkdir -p /content/TotoroUI/output/exp_data

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/LivePortrait_InsightFace/resolve/main/liveportrait/human/appearance_feature_extractor.safetensors -d /content/TotoroUI/models/liveportrait -o appearance_feature_extractor.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/LivePortrait_InsightFace/resolve/main/liveportrait/human/landmark.onnx -d /content/TotoroUI/models/liveportrait -o landmark.onnx
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/LivePortrait_InsightFace/resolve/main/liveportrait/human/landmark_model.pth -d /content/TotoroUI/models/liveportrait -o landmark_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/LivePortrait_InsightFace/resolve/main/liveportrait/human/motion_extractor.safetensors -d /content/TotoroUI/models/liveportrait -o motion_extractor.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/LivePortrait_InsightFace/resolve/main/liveportrait/human/spade_generator.safetensors -d /content/TotoroUI/models/liveportrait -o spade_generator.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/LivePortrait_InsightFace/resolve/main/liveportrait/human/stitching_retargeting_module.safetensors -d /content/TotoroUI/models/liveportrait -o stitching_retargeting_module.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/LivePortrait_InsightFace/resolve/main/liveportrait/human/warping_module.safetensors -d /content/TotoroUI/models/liveportrait -o warping_module.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/LivePortrait_InsightFace/resolve/main/test.png -d /content -o test.png

%cd /content/TotoroUI

from PIL import Image
import numpy as np
import nodes
from nodes import NODE_CLASS_MAPPINGS
from nodes import load_custom_node

load_custom_node("/content/TotoroUI/custom_nodes/TotoroUI-AdvancedLivePortrait")
LoadImage = NODE_CLASS_MAPPINGS["LoadImage"]()
ExpressionEditor = NODE_CLASS_MAPPINGS["ExpressionEditor"]()

/content
Cloning into '/content/TotoroUI'...
remote: Enumerating objects: 14631, done.
remote: Counting objects: 100% (3007/3007), done.
remote: Compressing objects: 100% (304/304), done.
remote: Total 14631 (delta 2821), reused 2758 (delta 2703), pack-reused 11624 (from 1)
Receiving objects: 100% (14631/14631), 23.06 MiB | 16.20 MiB/s, done.
Resolving deltas: 100% (9824/9824), done.
Cloning into '/content/TotoroUI/custom_nodes/TotoroUI-AdvancedLivePortrait'...
remote: Enumerating objects: 258, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 258 (delta 44), reused 43 (delta 32), pack-reused 184 (from 1)
Receiving objects: 100% (258/258), 14.18 MiB | 15.87 MiB/s, done.
Resolving deltas: 100% (111/111), done.
/content/TotoroUI
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import gradio as gr
from PIL import Image
import numpy as np

def generate_expression(image, rotate_pitch, rotate_yaw, rotate_roll, blink, eyebrow, wink, pupil_x, pupil_y, aaa, eee, woo, smile):
    # Load the uploaded image or default test image
    if image is None:
        image = '/content/test.png'
    src_image = LoadImage.load_image(image)[0]

    src_ratio = 1.0
    sample_ratio = 1.0
    sample_parts = "OnlyExpression"
    crop_factor = 1.7

    # Run the expression editor with user settings
    output_image = ExpressionEditor.run(rotate_pitch, rotate_yaw, rotate_roll, blink, eyebrow, wink, pupil_x, pupil_y, aaa, eee, woo, smile,
                                        src_ratio, sample_ratio, sample_parts, crop_factor, src_image=src_image, sample_image=None, motion_link=None, add_exp=None)

    # Convert the output to a PIL image
    output_np = np.array(output_image['result'][0] * 255, dtype=np.uint8)
    output_img = Image.fromarray(output_np[0])
    return output_img

# Define the Gradio interface with an upload option for images
iface = gr.Interface(
    fn=generate_expression,
    inputs=[
        gr.Image(type="filepath", label="Upload Image"),  # Image upload field
        gr.Slider(-20, 20, value=0, label="Rotate Pitch"),
        gr.Slider(-20, 20, value=0, label="Rotate Yaw"),
        gr.Slider(-20, 20, value=0, label="Rotate Roll"),
        gr.Slider(-20, 5, value=0, label="Blink"),
        gr.Slider(-10, 15, value=0, label="Eyebrow"),
        gr.Slider(0, 25, value=0, label="Wink"),
        gr.Slider(-15, 15, value=0, label="Pupil X"),
        gr.Slider(-15, 15, value=0, label="Pupil Y"),
        gr.Slider(-30, 120, value=0, label="AAA"),
        gr.Slider(-20, 15, value=0, label="EEE"),
        gr.Slider(-20, 15, value=0, label="WOO"),
        gr.Slider(-0.3, 1.3, value=1.3, label="Smile"),
    ],
    outputs=gr.Image(type="pil", label="Output Image"),
    title="Advanced Live Portrait Expression Editor"
)

# Launch the Gradio interface with sharing enabled
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ce64ab27f682e02c55.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
